In [12]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [13]:
import numpy as np
import pyprind
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pprint
import copy
import wurst as w
import brightway2 as bw
import bw2analyzer as bwa

from brightway2 import *
from matplotlib import pyplot as plt
from pandas import DataFrame
from openpyxl import load_workbook
from wurst import *
from wurst.searching import *
from wurst.transformations.activity import change_exchanges_by_constant_factor
from wurst.transformations.uncertainty import rescale_exchange
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from wurst.IMAGE.io import *
from wurst.IMAGE import *
from wurst.ecoinvent.electricity_markets import *
from wurst.ecoinvent.filters import *
from wurst.transformations.geo import *

## 1. Electricity production/mix from 2020 to 2100 (IMAGE results)

###### This results is the annual electricity production/mix results at different regions, we presented them (only electricity mix) in Figure 2, Figure 3a, and SI-Figure 2-5. The results includes SSP2-Baseline, SSP2-RCP1.9 w/o DACCS, and SSP2-RCP1.9 w/ DACCS scenarios  

In [ ]:
tech_sort = ['Coal_ST', 
             'IGCC',
             'Coal_CHP', 
             'Coal_CCS', 
             'Coal_CHP_CCS', 
             'Natural_gas_CC', 
             'Natural_gas_OC',
             'Natural_gas_CHP', 
             'Natural_gas_CCS',
             'Natural_gas_CHP_CCS', 
             'Oil_ST', 
             'Oil_CC', 
             'Oil_CHP', 
             'Oil_CCS', 
             'Oil_CHP_CCS', 
             'Nuclear', 
             'Hydro', 
             'Wind_onshore', 
             'Wind_offshore',
             'CSP', 
             'Solar_PV_decen', 
             'Solar_PV_cen', 
             'Biomass_ST', 
             'Biomass_CC', 
             'Biomass_CHP', 
             'Biomass_CCS',
             'Biomass_CHP_CCS', 
             'Wave', 
             'Other_renewables']

In [ ]:
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
year

In [ ]:
## Import the region information of IMAGE
IMAGE_variables = pd.read_excel('IMAGE_Data/Image variable names.xlsx')
REGIONS = IMAGE_variables['Regions'].dropna().values

## Depending on the scenatio we work on, we can activate different dir of the IMAGE results.
fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2\ElecProdSpec.out"
#fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19\ElecProdSpec.out"
#fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\ElecProdSpec.out"

elec_production = load_image_data_file(fp)

elec_prod_dfs = {}
for i, region in enumerate(REGIONS):
    elec_prod_dfs[region] = pd.DataFrame(elec_production.data[i,:,:],columns = elec_production.years,index = IMAGE_variables['Technology'].dropna().values).T.drop('EMPTY CATEGORY!!',axis = 1)
    ## Convert GJ to TWh.
    elec_prod_dfs[region] = elec_prod_dfs[region] * 277.778/1000000000 


def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
elec_prod_year = {}
elec_mix_year = {}
for y in year:
    elec_prod_year[y] = pd.concat([pd.Series(elec_prod_dfs[region].loc[y], name = region) for region in REGIONS[:-1]], axis = 1)

## Calculate the electricity mix (actual production)
elec_prod_year_summary_table = pd.concat(elec_prod_year ,axis=0)
elec_prod_year_summary_table.index.names = ["Year", 'Tech']
elec_prod_year_summary_table["World"] = elec_prod_year_summary_table.sum(axis=1)
elec_prod_year_summary_table = elec_prod_year_summary_table.reset_index()
elec_prod_export_table = elec_prod_year_summary_table[['Year', 'Tech', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()


## Calculate the electricity mix (Percentage)
elec_mix_year = {}
#year = [2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]
for y in year:
    elec_prod_one_year = elec_prod_year_summary_table[elec_prod_year_summary_table["Year"] == y]
    elec_prod_one_year = elec_prod_one_year.set_index(["Year", 'Tech'])
    elec_mix_year[y] = elec_prod_one_year.divide(elec_prod_one_year.sum(axis = 0))

elec_mix_year_summary_table = pd.concat(elec_mix_year ,axis=0)
elec_mix_year_summary_table = elec_mix_year_summary_table.reset_index()
elec_mix_export_table = elec_mix_year_summary_table[['Year', 'Tech', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()

## We calculated the results for both electricity production and mix, but we only presented electricity mix results in our manuscript.
#print(elec_prod_export_table)
#print(elec_mix_export_table)

In [ ]:
## Depending on the scenarios under which we calculate the electriciyt mix/production resutls, we activate different following "to_excel" function to export results as excel file. 

## Bring the electricity mix/production (Baseline) to excel
elec_prod_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_prod_summary_SSP2_baseline.xlsx', index = True)
elec_mix_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_mix_summary_SSP2_baseline.xlsx', index = True)

## Bring the electricity mix/production (SSP2-RCP1.9 w/o DACCS) to excel
#elec_prod_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_prod_summary_SSP2_RCP19.xlsx', index = True)
#elec_mix_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_mix_summary_SSP2_RCP19.xlsx', index = True)

## Bring the electricity mix/production (SSP2-RCP1.9 w/ DACCS) to excel
#elec_prod_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_prod_summary_SSP2_RCP19_DAC.xlsx', index = True)
#elec_mix_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_mix_summary_SSP2_RCP19_DAC.xlsx', index = True)

## 2. Electricity CO2 emission from 2020 to 2100 (IMAGE results)
###### This results are presented in Figure 2d, and SI-Figure 2d, 3d, 4d, and 5d. The results includes SSP2-Baseline, SSP2-RCP1.9 w/o DACCS, and SSP2-RCP1.9 w/ DACCS scenarios  

In [16]:
IMAGE_variables = pd.read_excel('IMAGE_Data/Image variable names.xlsx')
REGIONS = IMAGE_variables['Regions'].dropna().values

## We need to make specific "CO2_region_list" for the for loop later, because Index No.27 (world) are all 0 in the CO2 data, while Index No.28 (world2) represent the CO2 emission data at the world level
CO2_region_list = REGIONS.tolist()
CO2_region_list.append('World2')
#CO2_region_list

In [ ]:
## The dataset has three dimensions, Regions * emitting_sector * primary_energy_carrier. Here, we include the information for the other two dimensions (emitting_sector and primary_energy_carrier)

## The dimention information of IMAGECO2 data
emitting_sector = [
    'Industry', 
    'Transport', 
    'Residential', 
    'Services', 
    'Other_Energy_demand', 
    'Electricity', 
    'Heat', 
    'Hydrogen_production', 
    'Flaring', 
    'Eroi', 
    'Cokes', 
    'Bioenergy_production', 
    'BECCS_in_bioliquid_production', 
    'Cement_Production', 
    'Chemical_production', 
    'Statistical_differences_1'
    'Statistical_differences_2', 
    'LUC_estimate', 
    'Total',
    'unknown19'
]

## Not sure what is "unknown_6"
primary_energy_carrier = [
    'Coal', 
    'Heavy_Oil', 
    'Light_Oil', 
    'Natural_Gas',
    'biofuel',
    'total'
]

In [ ]:
## Import the IMAGE .out file that includes the CO2 emission data
## Depending on the scenatio we work on, we can activate different dir of the IMAGE results.

fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2\CO2Spec2.out"
#fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19\CO2Spec2.out"
#fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\CO2Spec2.out"

elec_co2_emission = load_image_data_file(fp)


## Extract CO2 data from "elec_co2_emission", and assign the data with country name, also we only extract electricity CO2 emission data by specifying index "5" from "emitting_sector" 
elec_co2_dfs = {}
for i, region in enumerate(CO2_region_list):
    elec_co2_dfs[region] = pd.DataFrame(elec_co2_emission.data[i,5,:],columns = elec_co2_emission.years, index = primary_energy_carrier)
    elec_co2_dfs[region] = elec_co2_dfs[region] * (44/12) 


## Create an empty dataframe to summarize the region CO2 data
elec_co2_region_sum_df = pd.DataFrame(columns = elec_co2_emission.years)
elec_co2_region_sum_df 


## Calculate the country annual total CO2 data
for region in CO2_region_list:
    a = elec_co2_dfs[region].loc[['total']]
    elec_co2_region_sum_df = elec_co2_region_sum_df.append(a)


## Assign each row with country name
elec_co2_region_sum_df.index = CO2_region_list
elec_co2_region_sum_df_final = elec_co2_region_sum_df.drop('World')
elec_co2_region_sum_df_final


## Select the data from 2020 to 2100 
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
elec_co2_region_sum_df_final_2020_2100 = elec_co2_region_sum_df_final[year]
elec_co2_region_sum_df_final_2020_2100


## Select the five regions needed, tanspose the data, rename the column name.
elec_co2_5_region = elec_co2_region_sum_df_final_2020_2100.loc[['China Region', 'USA', 'Western Europe', 'Russia Region', 'World2']]
elec_co2_5_region_export = elec_co2_5_region.T.reset_index()
elec_co2_5_region_export.columns =['Year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']
elec_co2_5_region_export

In [ ]:
## Depending on the scenarios under which we calculate the electricity CO2 resutls, we activate different following "to_excel" function to export results as excel file. 


## export the electricity CO2 (Baseline) to excel
elec_co2_5_region_export.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_CO2_summary_SSP2_baseline.xlsx', index = True)

## export the electricity CO2 (RCP19) to excel
#elec_co2_5_region_export.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_CO2_summary_SSP2_RCP19.xlsx', index = True)

## export the electricity CO2 (RCP19_DAC) to excel
#elec_co2_5_region_export.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\elec_CO2_summary_SSP2_RCP19_DAC.xlsx', index = True)

## 3. DACCS electricity demand from 2020 to 2100 (IMAGE results)
##### The DACCS electricity demand is presented in Figure 2 and SI-Figure 2-5. This result is only available for SSP2-RCP1.9 w/ DACCS scenairo


In [ ]:
elec_demand_tech = ['Industry',
                    'Transport',
                    'Residential',
                    'Services',
                    'Other',
                    'Cokes',
                    'DAC',
                    'H2',
                    'Bunkers',
                    'EROI']

In [ ]:
# the dir of IMAGE output of electricity demand
elec_demand_path = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\ElecNetDemand.out"

## Import the electricity demand data
elec_demand = load_image_data_file(elec_demand_path)

In [ ]:
elec_demand_dfs = {}

for i, region in enumerate(REGIONS[:-1]):
    elec_demand_dfs[region] = pd.DataFrame(elec_demand.data[i,:,:],columns = elec_demand.years,index = elec_demand_tech).T
    ## Convert GJ to TWh.
    elec_demand_dfs[region] = elec_demand_dfs[region] * 277.778/1000000000
    
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
elec_demand_year = {}

for y in year:
    elec_demand_year[y] = pd.concat([pd.Series(elec_demand_dfs[region].loc[y], name = region) for region in REGIONS[:-1]], axis = 1)

In [ ]:
elec_demand_year_summary_table = pd.concat(elec_demand_year ,axis=0)
elec_demand_year_summary_table.index.names = ["Year", 'Tech']
elec_demand_year_summary_table["World"] = elec_demand_year_summary_table.sum(axis=1)
elec_demand_year_summary_table = elec_demand_year_summary_table.reset_index()
elec_demand_export_table = elec_demand_year_summary_table[['Year', 'Tech', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].loc[lambda row : row['Tech'] == 'DAC'].copy()
elec_demand_export_table

In [ ]:
elec_demand_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\DACCS_elect_demand_data.xlsx', index = True, sheet_name='DAC_elect_demand')

## 4. Cumulative installed DAC capacity (global level) and annual operational DAC capacity (global level) from 2020 to 2100 (IMAGE results)
##### The cumulative installed DAC capacity is used to calculate the future material and energy consumption using learning curve approach
##### The operational DAC capacity is presented in Figure.3 

In [19]:
# IMAGE data of annual new installed capacity of DAC
DAC_fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\CapInvDAC.out"

# IMAGE data of annual operational capacity of DAC
DAC_op_fp = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\CapDAC.out"

#IMAGE_variables = pd.read_excel('IMAGE_Data/Image variable names.xlsx')
#REGIONS = IMAGE_variables['Regions'].dropna().values

In [24]:
## Import the annual new installed capacity, select the year from 2020 to 2100, and store the data into excel 
DAC_deployment = load_image_data_file(DAC_fp)
DAC_deploy_data = pd.DataFrame(DAC_deployment.data[:,:],columns = DAC_deployment.years).drop(26).set_index(REGIONS).loc[:, '2020':'2100']
DAC_deploy_data.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Learning curve\DAC_deployment_result.xlsx', index = True, sheet_name='annual_deploy_Gt')
DAC_deploy_data

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054337,0.054915,0.054344,0.053540,0.053669,0.052189,0.051793,0.052259,0.052313,0.052423
USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050507,0.050753,0.050812,0.049878,0.050111,0.050035,0.050722,0.051960,0.052381,0.052814
Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003025,0.003207,0.003172,0.003401,0.003457,0.003426,0.003448,0.003497,0.003374,0.003356
Central America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005386,0.005867,0.005947,0.006594,0.006836,0.006975,0.007192,0.007412,0.007478,0.007614
Brazil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.031771,0.031955,0.032057,0.032166,0.032240,0.032333,0.032386,0.032477,0.032483,0.032399
Rest of South America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.039851,0.040101,0.040324,0.040543,0.040742,0.040920,0.041067,0.041195,0.041268,0.041215
Northern Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.053677,0.053329,0.053021,0.052864,0.052858,0.052949,0.053093,0.053199,0.053302,0.053315
Western Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.057543,0.057754,0.057752,0.057535,0.057357,0.056715,0.056012,0.055239,0.054691,0.054176
Eastern Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
South Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007145,0.007342,0.007271,0.007320,0.007433,0.007475,0.007525,0.007517,0.007486,0.007411


In [25]:
## Calculate the cumulative installed capacity of DAC at the world level from 2020 to 2100, and store it in the excel.
DAC_deploy_data_cum = DAC_deploy_data.cumsum(axis=1)
World_DAC_cum_data = DAC_deploy_data_cum.loc[['World']]

book = load_workbook(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Learning curve\DAC_deployment_data.xlsx')
writer = pd.ExcelWriter(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Learning curve\DAC_deployment_result.xlsx', engine = 'openpyxl')
writer.book = book
World_DAC_cum_data.to_excel(writer, index = True, sheet_name='World_DAC_cum_data_Gt')
writer.save()


World_DAC_cum_data

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
World,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.576419,15.345556,16.115884,16.886406,17.656919,18.420596,19.17661,19.917299,20.638685,21.342531


In [26]:
## Import the annual operational DAC capacity, select the year from 2020 to 2100, and store the data into excel 
DAC_operation = load_image_data_file(DAC_op_fp)
DAC_oper_data = pd.DataFrame(DAC_operation.data[:,:],columns = DAC_operation.years).drop(26).set_index(REGIONS).loc[:, '2020':'2100']

book = load_workbook(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Learning curve\DAC_deployment_data.xlsx')
writer = pd.ExcelWriter(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Learning curve\DAC_deployment_result.xlsx', engine = 'openpyxl')
writer.book = book
DAC_oper_data.to_excel(writer, index = True, sheet_name='DAC_operation_Gt')
writer.save()

DAC_oper_data

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.812618,0.850944,8.863843e-01,9.188790e-01,9.481778e-01,9.740645e-01,9.965657e-01,1.015228e+00,1.030004e+00,1.041344e+00
USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.529218,0.569518,6.093699e-01,6.484099e-01,6.862746e-01,7.224882e-01,7.567910e-01,7.887033e-01,8.182175e-01,8.454954e-01
Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.039222,0.041560,4.377842e-02,4.590890e-02,4.795545e-02,4.995158e-02,5.188146e-02,5.376114e-02,5.554944e-02,5.725962e-02
Central America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.046161,0.050841,5.555182e-02,6.032145e-02,6.512921e-02,7.003495e-02,7.500970e-02,8.011702e-02,8.528246e-02,9.051722e-02
Brazil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.388523,0.413921,4.384193e-01,4.618396e-01,4.839700e-01,5.046474e-01,5.237311e-01,5.411525e-01,5.568721e-01,5.709142e-01
Rest of South America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.554916,0.584198,6.118670e-01,6.376981e-01,6.615034e-01,6.831554e-01,7.025737e-01,7.197742e-01,7.348300e-01,7.478750e-01
Northern Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.894156,0.923571,9.497079e-01,9.724708e-01,9.918647e-01,1.008017e+00,1.021177e+00,1.031695e+00,1.039997e+00,1.046500e+00
Western Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.857689,0.896024,9.313922e-01,9.637213e-01,9.928807e-01,1.018767e+00,1.041377e+00,1.060710e+00,1.076831e+00,1.089889e+00
Eastern Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.023569,0.022519,2.125392e-02,1.974493e-02,1.801022e-02,1.609949e-02,1.407201e-02,1.200158e-02,9.977877e-03,8.058329e-03
South Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.077322,0.082958,8.855543e-02,9.408727e-02,9.944291e-02,1.045798e-01,1.094555e-01,1.140295e-01,1.182622e-01,1.221658e-01


## 5. Primary fossil fuel consumption from 2020 to 2100 (IMAGE results)
###### This results are used to make SI-Figure 7

In [ ]:
# the path of primary energy data (under SSP2-RCP1.9 w/o DACCS and SSP2-RCP1.9 w/ DACCS scenarios)
PE_use_RCP19_path = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19\tpes.out"
PE_use_RCP19_DAC_path = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\tpes.out"

## Import the primary energy data
PE_use_RCP19 = load_image_data_file(PE_use_RCP19_path)
PE_use_RCP19_DAC = load_image_data_file(PE_use_RCP19_DAC_path)

In [ ]:
Fuel_type = [
   'Coal',
'Conventional Oil',
'Unconventional Oil',
'Natural Gas',
'Modern Biofuels',
'Traditional Biofuels',
'Nuclear',
'Solar/Wind',
'Hydroelectricity',
'Total' 
]

In [ ]:
PE_use_RCP19_df = {}

#Unit is PJ = 1,000,000 GJ
for i, region in enumerate(REGIONS[:-1]):
    PE_use_RCP19_df[region] = pd.DataFrame(PE_use_RCP19.data[i,:,:], columns = PE_use_RCP19.years, index = Fuel_type).T
    
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
NG_use_year = {}

for y in year:
    PE_use_year[y] = pd.concat([pd.Series(PE_use_RCP19_df[region].loc[y], name = region) for region in REGIONS[:-1]], axis = 1)
    
PE_use_year_summary_table = pd.concat(PE_use_year ,axis=0)
PE_use_year_summary_table.index.names = ["Year", 'Tech']
PE_use_year_summary_table["World"] = PE_use_year_summary_table.sum(axis=1)
PE_use_year_summary_table = PE_use_year_summary_table.reset_index()
PE_use_RCP19_export_table = PE_use_year_summary_table[['Year', 'Tech', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()

PE_use_RCP19_export_table
PE_use_RCP19_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\PE_use_RCP19_data.xlsx', index = True, sheet_name='PE_use_RCP19')

In [ ]:
PE_use_RCP19_DAC_df = {}
PE_use_RCP19_DAC
#Unit is PJ = 1,000,000 GJ
for i, region in enumerate(REGIONS[:-1]):
    PE_use_RCP19_DAC_df[region] = pd.DataFrame(PE_use_RCP19_DAC.data[i,:,:], columns = PE_use_RCP19_DAC.years, index = Fuel_type).T
    
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
PE_use_year = {}

for y in year:
    PE_use_year[y] = pd.concat([pd.Series(PE_use_RCP19_DAC_df[region].loc[y], name = region) for region in REGIONS[:-1]], axis = 1)
    
PE_use_year_summary_table = pd.concat(PE_use_year ,axis=0)
PE_use_year_summary_table.index.names = ["Year", 'Tech']
PE_use_year_summary_table["World"] = PE_use_year_summary_table.sum(axis=1)
PE_use_year_summary_table = PE_use_year_summary_table.reset_index()
PE_use_RCP19_DAC_export_table = PE_use_year_summary_table[['Year', 'Tech', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()

PE_use_RCP19_DAC_export_table
PE_use_RCP19_DAC_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\PE_use_RCP19_DAC_data.xlsx', index = True, sheet_name='PE_use_RCP19_DAC')

###### The exported results here includes all types of fuel. We post-processed these results to includes only fossil fuels (Natural gas, Coal, and Conventional and Unconventional Oil when we create the figure in R-scripts)

## 6. CO2 sequestration results from 2020 to 2100 (IMAGE results)
###### This results are used to make SI-Figure 8

In [ ]:
# the path of CO2 sequestration data (under SSP2-RCP1.9 w/o DACCS and SSP2-RCP1.9 w/ DACCS scenarios)
carbon_sequ_RCP19_path = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19\CarbonCapturedSpec2.out"
carbon_sequ_RCP19_DAC_path = r"C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\Work\WURST_code\IMAGE_Data\SSP2_RCP19_DAC\CarbonCapturedSpec2.out"

## Import the CO2 sequestration data
carbon_sequ_RCP19 = load_image_data_file(carbon_sequ_RCP19_path)
carbon_sequ_RCP19_DAC = load_image_data_file(carbon_sequ_RCP19_DAC_path)

In [ ]:
Fuels_type = [
    'Coal',
    'Oil',
    'Natural Gas',
    'Biomass',
    'Other',
    'Total' ]

Sector = [
    'Electricity',
    'Industry (energy)',
    'Cement',
    'Hydrogen',
    'Liquid biofuel production',
    'Transformation',
    'Heat production',
    'DAC']

##### SSP2-RCP_19 w/o DACCS scenario

In [ ]:
## DACCS results

## The dataset has three dimensions, Regions * Sector * Fuel_types. 
## In this step we get CO2 sequestration results from DACCS only by specifying index as "7" in the sector dimension.

## The Unit is 1 Petagram (equal to 1 Gt)
DAC_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    DAC_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19.data[i,7,:],columns = carbon_sequ_RCP19.years, index = Fuels_type)
    DAC_co2_seq_dfs[region] = DAC_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
DAC_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19.years)
DAC_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = DAC_co2_seq_dfs[region].loc['Total']
    DAC_co2_seq_sum_df = DAC_co2_seq_sum_df.append(a)
    
DAC_co2_seq_sum_df.index = REGIONS[:-1]

## Select the data from 2020 to 2100 
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
DAC_co2_seq_sum_df_final_2020_2100 = DAC_co2_seq_sum_df[year]


DAC_co2_seq_sum_df_trans = DAC_co2_seq_sum_df_final_2020_2100.T.copy()
DAC_co2_seq_sum_df_trans.index.names = ['year']
DAC_co2_seq_sum_df_trans["World"] = DAC_co2_seq_sum_df_trans.sum(axis=1)
DAC_co2_seq_sum_df_trans = DAC_co2_seq_sum_df_trans.reset_index()
DAC_co2_seq_RCP19_export_table = DAC_co2_seq_sum_df_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#DAC_co2_seq_export_table

DAC_co2_seq_RCP19_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\DAC_co2_seq_RCP19_data.xlsx', index = True, sheet_name='DAC_co2_seq')

In [ ]:
## BECCS results

## In this step we get CO2 sequestration results from BECCS only by specifying index as "3" in the Fuels_type dimension. 

## The Unit is 1 Petagram (equal to 1 Gt)
BECCS_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    BECCS_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19.data[i,:,3],columns = carbon_sequ_RCP19.years, index = Sector)
    BECCS_co2_seq_dfs[region] = BECCS_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
BECCS_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19.years)
BECCS_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = BECCS_co2_seq_dfs[region]
    a['region'] = region
    BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.append(a)

BECCS_co2_seq_sum_df.index.names = ['sector'] 
BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.reset_index()
BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.set_index(['region', 'sector'])

def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
BECCS_co2_seq_sum_df_final_2020_2100 = BECCS_co2_seq_sum_df[year]
#BECCS_co2_seq_sum_df_final_2020_2100

BECCS_co2_seq_region_sum = BECCS_co2_seq_sum_df_final_2020_2100.groupby(level = 0).sum()
#BECCS_co2_seq_region_sum

BECCS_co2_seq_region_sum_trans = BECCS_co2_seq_region_sum.T.copy()
BECCS_co2_seq_region_sum_trans.index.names = ['year']
BECCS_co2_seq_region_sum_trans["World"] = BECCS_co2_seq_region_sum_trans.sum(axis=1)
BECCS_co2_seq_region_sum_trans = BECCS_co2_seq_region_sum_trans.reset_index()
BECCS_co2_seq_RCP19_export_table = BECCS_co2_seq_region_sum_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#BECCS_co2_seq_export_table
BECCS_co2_seq_RCP19_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\BECCS_co2_seq_RCP19_data.xlsx', index = True, sheet_name='BECCS_co2_seq')

In [ ]:
## Total CO2 sequestration results

## In this step we get total CO2 sequestration results by specifying index as "5" in the Fuels_type dimension. By substracting CO2 sequestration of BECCS and DACCS from the total CO2 sequestration, we can get
## the CO2 sequestration from all othter technologies (this calculation is done when we made the SI-figure 8 using R-scripts)

## The Unit is 1 Petagram (equal to 1 Gt)
total_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    total_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19.data[i,:,5],columns = carbon_sequ_RCP19.years, index = Sector)
    total_co2_seq_dfs[region] = total_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
total_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19.years)
#total_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = total_co2_seq_dfs[region]
    a['region'] = region
    total_co2_seq_sum_df = total_co2_seq_sum_df.append(a)

total_co2_seq_sum_df.index.names = ['sector'] 
total_co2_seq_sum_df = total_co2_seq_sum_df.reset_index()
total_co2_seq_sum_df = total_co2_seq_sum_df.set_index(['region', 'sector'])

def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
total_co2_seq_sum_df_final_2020_2100 = total_co2_seq_sum_df[year]
#total_co2_seq_sum_df_final_2020_2100

total_co2_seq_region_sum = total_co2_seq_sum_df_final_2020_2100.groupby(level = 0).sum()
#total_co2_seq_region_sum

total_co2_seq_region_sum_trans = total_co2_seq_region_sum.T.copy()
total_co2_seq_region_sum_trans.index.names = ['year']
total_co2_seq_region_sum_trans["World"] = total_co2_seq_region_sum_trans.sum(axis=1)
total_co2_seq_region_sum_trans = total_co2_seq_region_sum_trans.reset_index()
total_co2_seq_RCP19_export_table = total_co2_seq_region_sum_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#total_co2_seq_export_table
total_co2_seq_RCP19_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\total_co2_seq_RCP19_data.xlsx', index = True, sheet_name='total_co2_seq')

##### SSP2-RCP_19 w/ DACCS scenario

In [ ]:
## DACCS data

## The dataset has three dimensions, Regions * Sector * Fuel_types. 
## In this step we get CO2 sequestration results from DACCS only by specifying index as "7" in the sector dimension.

## The Unit is 1 Petagram (equal to 1 Gt)
DAC_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    DAC_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19_DAC.data[i,7,:],columns = carbon_sequ_RCP19_DAC.years, index = Fuels_type)
    DAC_co2_seq_dfs[region] = DAC_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
DAC_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19_DAC.years)
DAC_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = DAC_co2_seq_dfs[region].loc['Total']
    DAC_co2_seq_sum_df = DAC_co2_seq_sum_df.append(a)
    
DAC_co2_seq_sum_df.index = REGIONS[:-1]

## Select the data from 2020 to 2100 
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
DAC_co2_seq_sum_df_final_2020_2100 = DAC_co2_seq_sum_df[year]


DAC_co2_seq_sum_df_trans = DAC_co2_seq_sum_df_final_2020_2100.T.copy()
DAC_co2_seq_sum_df_trans.index.names = ['year']
DAC_co2_seq_sum_df_trans["World"] = DAC_co2_seq_sum_df_trans.sum(axis=1)
DAC_co2_seq_sum_df_trans = DAC_co2_seq_sum_df_trans.reset_index()
DAC_co2_seq_RCP19_DAC_export_table = DAC_co2_seq_sum_df_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#DAC_co2_seq_export_table

DAC_co2_seq_RCP19_DAC_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\DAC_co2_seq_RCP19_DAC_data.xlsx', index = True, sheet_name='DAC_co2_seq')

In [ ]:
## BECCS data

## In this step we get CO2 sequestration results from BECCS only by specifying index as "3" in the Fuels_type dimension. 

## The Unit is 1 Petagram (equal to 1 Gt)
BECCS_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    BECCS_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19_DAC.data[i,:,3],columns = carbon_sequ_RCP19_DAC.years, index = Sector)
    BECCS_co2_seq_dfs[region] = BECCS_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
BECCS_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19_DAC.years)
BECCS_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = BECCS_co2_seq_dfs[region]
    a['region'] = region
    BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.append(a)

BECCS_co2_seq_sum_df.index.names = ['sector'] 
BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.reset_index()
BECCS_co2_seq_sum_df = BECCS_co2_seq_sum_df.set_index(['region', 'sector'])

def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
BECCS_co2_seq_sum_df_final_2020_2100 = BECCS_co2_seq_sum_df[year]
#BECCS_co2_seq_sum_df_final_2020_2100

BECCS_co2_seq_region_sum = BECCS_co2_seq_sum_df_final_2020_2100.groupby(level = 0).sum()
#BECCS_co2_seq_region_sum

BECCS_co2_seq_region_sum_trans = BECCS_co2_seq_region_sum.T.copy()
BECCS_co2_seq_region_sum_trans.index.names = ['year']
BECCS_co2_seq_region_sum_trans["World"] = BECCS_co2_seq_region_sum_trans.sum(axis=1)
BECCS_co2_seq_region_sum_trans = BECCS_co2_seq_region_sum_trans.reset_index()
BECCS_co2_seq_RCP19_DAC_export_table = BECCS_co2_seq_region_sum_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#BECCS_co2_seq_export_table
BECCS_co2_seq_RCP19_DAC_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\BECCS_co2_seq_RCP19_DAC_data.xlsx', index = True, sheet_name='BECCS_co2_seq')

In [ ]:
## Total seq data

## In this step we get total CO2 sequestration results by specifying index as "5" in the Fuels_type dimension. By substracting CO2 sequestration of BECCS and DACCS from the total CO2 sequestration, we can get
## the CO2 sequestration from all othter technologies (this calculation is done when we made the SI-figure 8 using R-scripts)

## The Unit is 1 Petagram (equal to 1 Gt)
total_co2_seq_dfs = {}
for i, region in enumerate(REGIONS[:-1]):
    total_co2_seq_dfs[region] = pd.DataFrame(carbon_sequ_RCP19_DAC.data[i,:,5],columns = carbon_sequ_RCP19_DAC.years, index = Sector)
    total_co2_seq_dfs[region] = total_co2_seq_dfs[region] * (44/12) 
    
## Create an empty dataframe to summarize the region CO2 data
total_co2_seq_sum_df = pd.DataFrame(columns = carbon_sequ_RCP19_DAC.years)
#total_co2_seq_sum_df 

## Calculate the country annual total CO2 data
for region in REGIONS[:-1]:
    a = total_co2_seq_dfs[region]
    a['region'] = region
    total_co2_seq_sum_df = total_co2_seq_sum_df.append(a)

total_co2_seq_sum_df.index.names = ['sector'] 
total_co2_seq_sum_df = total_co2_seq_sum_df.reset_index()
total_co2_seq_sum_df = total_co2_seq_sum_df.set_index(['region', 'sector'])

def createList(r1, r2):
    return [item for item in range(r1, r2+1)]
year = createList(2020, 2100)
total_co2_seq_sum_df_final_2020_2100 = total_co2_seq_sum_df[year]
#total_co2_seq_sum_df_final_2020_2100

total_co2_seq_region_sum = total_co2_seq_sum_df_final_2020_2100.groupby(level = 0).sum()
#total_co2_seq_region_sum

total_co2_seq_region_sum_trans = total_co2_seq_region_sum.T.copy()
total_co2_seq_region_sum_trans.index.names = ['year']
total_co2_seq_region_sum_trans["World"] = total_co2_seq_region_sum_trans.sum(axis=1)
total_co2_seq_region_sum_trans = total_co2_seq_region_sum_trans.reset_index()
total_co2_seq_RCP19_DAC_export_table = total_co2_seq_region_sum_trans[['year', 'China Region', 'USA', 'Western Europe', 'Russia Region', 'World']].copy()
#total_co2_seq_export_table
total_co2_seq_RCP19_DAC_export_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_mix\total_co2_seq_RCP19_DAC_data.xlsx', index = True, sheet_name='total_co2_seq')